# Sudoku Solver

Taro Kuriyama, January 2016

Functionally inspired by Richard Bird's solution in *Pearls of Functional Algorithm Design*.

<hr>

### Define and Show Board

The Sudoku board is provided as a string of digits, where either '0' or '.' defines a blank cell. 

This string is parsed into a list of 9 rows referred to throughout as `board`. Each row is a list comprised of 9 cells, where each cell is a set reflecting possible integer values. A pre-defined cell will be parsed into a singleton set, whereas an empty cell will be parsed into a set of all possible values {1, 2, 3, 4, 5, 6, 7, 8, 9}.

In [21]:
def group(items, n):
    """Group sequence into n-tuples."""
    return zip(*[items[i::n] for i in xrange(n)])

def parse(board_str):
    """Parse puzzle input string into board object."""
    assert len(board_str) == 81
    digits = {1, 2, 3, 4, 5, 6, 7, 8, 9}
    board = group(board_str, 9)
    return [[digits if n in ('0', '.') else set([int(n)]) for n in row]
            for row in board]

**Sample Boards**

Some sample boards are borrowed from http://www.7sudoku.com/instructions/loading-puzzles.

`Show` returns a string representation of the board that can be printed for visualization.

In [22]:
easy = parse('..6...94.9.....3....4.92...6.7.1..2.5.23.64.9.3..4.7.5...68.5....5.....4.98...1..')
medium = parse('...28.94.1.4...7......156.....8..57.4.......8.68..9.....196......5...8.3.43.28...')
hard = parse('...16..2...2...8.5..5..36.9....5.18...........96.7....1.89..3..4.9...7...5..16...')

In [23]:
def to_str(sets):
    """Convert sequence of integer sets to string."""
    return ' '.join(['.' if len(s) > 1 else str(tuple(s)[0])
                     for s in sets])
    
def show(board):
    """Convert board object to string representation."""
    bars = '-' * 21 + '\n'
    output = bars
    row_group = group(board, 3)
    for rows in row_group:
        for row in rows:
            output += ' | '.join([to_str(cols) for cols in group(row, 3)])
            output += '\n'
        output += bars
    return output

In [24]:
print show(easy)

---------------------
. . 6 | . . . | 9 4 .
9 . . | . . . | 3 . .
. . 4 | . 9 2 | . . .
---------------------
6 . 7 | . 1 . | . 2 .
5 . 2 | 3 . 6 | 4 . 9
. 3 . | . 4 . | 7 . 5
---------------------
. . . | 6 8 . | 5 . .
. . 5 | . . . | . . 4
. 9 8 | . . . | 1 . .
---------------------



### Board Validation

A board is valid if there are no duplicate numbers in any rows, columns, or boxes. 

A board is complete if **(a)** it is valid and **(b)** the sum of each row, column, or box is 45 (assuming only digits 0 - 9 are ever used in the board). Note that given (a), only one of rows, columns, or boxes need to be checked for (b).

The validation functions always iterate over board transformations, which are generated by the functions `rows` (identity function), `cols` (matrix transposition), and `boxs` (grouping into boxes). This keeps the validation logic simple.

Note that `rows`, `cols`, and `boxs` are all involutions for a 9 x 9 Sudoku board as defined here, so `cols(cols(board))` is equivalent to just `board`.

In [25]:
def rows(board):
    """Return board."""
    return board

def cols(board):
    """Return transposed board."""
    return zip(*board)

def flatten(nested):
    """Flatten nested sequence of sequences."""
    return list(n for sublist in nested for n in sublist)

def boxs(board):
    """Group board by its boxes.
    For each triple of rows, group each row into 3 digits and 
    zip up the rows; the resulting lists are the board boxes.
    """
    boxes = []
    for grouped in group(board, 3):
        triple = [group(row, 3) for row in grouped]
        zipped = zip(*triple)
        rows = [flatten(row) for row in zipped]
        boxes.extend(rows)
    return boxes

In [26]:
def singletons(row):
    """Return all singleton values from list of sets"""
    return [tuple(ns)[0] for ns in row if len(ns) == 1]

def noempties(board):
    """Iterate through board; return True if there are no
    empty cells in any row."""
    return all(ns for ns in flatten(board))

def nodups(board):
    """Iterate through board; return True if there are no 
    duplicate singleton numbers in any row.
    """
    checks = []
    for row in board:
        singles = singletons(row)
        checks.append(len(singles) == len(set(singles)))
    return all(checks)

def valid(board):
    """Return True if board is valid."""
    return (noempties(board) and 
            all(nodups(f(board)) for f in (rows, cols, boxs)))
    
def complete(board):
    """Return True if board is complete."""
    return (valid(board) and 
            all([sum(singletons(row)) == 45 for row in board]))

**Validate Sample Boards**

We can run some trivial unit tests to verify the input boards are as expected.

In [27]:
def verify_board(board):
    """Some basic unit tests"""
    assert valid(board) is True
    assert complete(board) is False
    assert len(board) == 9
    return True

In [28]:
print verify_board(easy)
print verify_board(medium)
print verify_board(hard)

True
True
True


<hr>

### Solver: Prune, Fill, Search

The solver works on a stack of boards and operates in a pipeline of three steps:
1. prune
2. fill
3. search

**Prune**

For each unit of row, column, or box, any cell values that are also singletons in the unit can be removed as possibilities.

**Fill**

For each unit of row, column, or box, if a value can only appear in one cell, fill that cell with the value.

**Search**

Find possible next boards by trying possible values for cells. It makes sense to work with cells that have the smallest number of possibilities. DFS is much faster than BFS for this solving model; invalid boards are pruned after search iteration.





In [29]:
from collections import defaultdict

In [30]:
def prune(board):
    """Remove known choices from the board."""
    rows = []
    for row in board:
        singles = singletons(row)
        new = [ns - set(singles) if len(ns) > 1 else ns
               for ns in row]
        rows.append(new)
    return rows

In [31]:
def fill(board):
    """Fill cells where only a single value is possible."""
    new_board = []
    for row in board:
        # count non-singleton digits in row
        digits = defaultdict(int)
        ns = [n for nums in row for n in nums 
              if len(nums) > 1 and n not in singletons(row)]
        for n in ns:
            digits[n] += 1
            
        # fill values that appear only once 
        new_row = list(row)
        for ind, nums in enumerate(row):
            for n in nums:
                if digits[n] == 1:
                    new_row = new_row[:ind] + [set([n])] + new_row[ind+1:]
                    break
        new_board.append(new_row)
    
    return new_board

In [32]:
def next_boards(board):
    """Generate list of some possible next boards."""
    flat = flatten(board)
    len_choices = [len(ns) for ns in flat if len(ns) > 1]
    
    if not len_choices: return []
    min_len = min(len_choices)

    boards = []
    for ind, ns in enumerate(flat):
        if len(ns) == min_len:
            flats = [flat[:ind] + [set([n])] + flat[ind+1:] for n in ns]
            boards.extend([group(board, 9) for board in flats])
            
    return boards

The solver itself is straightforward because `rows`, `cols`, `boxs` are involutions and allow `prune` and `fill` to iterate over board transformations. For each board transformation, the board is first pruned, then filled, then inverted back to its original arrangement.

In [33]:
def solve(board):
    """Solver: prune, fill, and search in order."""
    boards = [board]
    
    while boards:
        board = boards.pop()    
        for f in (rows, cols, boxs):
            board = prune(f(board))
            board = f(fill(board))
        
        if complete(board): 
            return show(board)
        if valid(board):
            boards.extend(next_boards(board))

    return []

### Results

The solver is designed with simplicity over performance in mind, but appears to work reasonably well. It can solve 17-hint puzzles (minimum uniquely completable) but takes a very long time or gets stuck on the ones reported by others as difficult.

In [34]:
print show(hard)
print solve(hard)

---------------------
. . . | 1 6 . | . 2 .
. . 2 | . . . | 8 . 5
. . 5 | . . 3 | 6 . 9
---------------------
. . . | . 5 . | 1 8 .
. . . | . . . | . . .
. 9 6 | . 7 . | . . .
---------------------
1 . 8 | 9 . . | 3 . .
4 . 9 | . . . | 7 . .
. 5 . | . 1 6 | . . .
---------------------

---------------------
9 8 3 | 1 6 5 | 4 2 7
6 1 2 | 7 9 4 | 8 3 5
7 4 5 | 2 8 3 | 6 1 9
---------------------
2 7 4 | 6 5 9 | 1 8 3
5 3 1 | 8 4 2 | 9 7 6
8 9 6 | 3 7 1 | 5 4 2
---------------------
1 6 8 | 9 2 7 | 3 5 4
4 2 9 | 5 3 8 | 7 6 1
3 5 7 | 4 1 6 | 2 9 8
---------------------



In [35]:
%%timeit
solve(easy)

100 loops, best of 3: 8.7 ms per loop


In [36]:
%%timeit
solve(medium)

100 loops, best of 3: 13.4 ms per loop


In [37]:
%%timeit
solve(hard)

1 loops, best of 3: 403 ms per loop


Some more puzzles from a massive repository of 17-hint puzzles collected here: http://staffhome.ecm.uwa.edu.au/~00013890/sudoku17

In [38]:
puzzles = """
000000010400000000020000000000050407008000300001090000300400200050100000000806000
000000010400000000020000000000050604008000300001090000300400200050100000000807000
000000012000035000000600070700000300000400800100000000000120000080000040050000600"""

In [39]:
for puzzle in puzzles.split('\n')[1:]:
    b = parse(puzzle)
    verify_board(b)
    print solve(b)

---------------------
6 9 3 | 7 8 4 | 5 1 2
4 8 7 | 5 1 2 | 9 3 6
1 2 5 | 9 6 3 | 8 7 4
---------------------
9 3 2 | 6 5 1 | 4 8 7
5 6 8 | 2 4 7 | 3 9 1
7 4 1 | 3 9 8 | 6 2 5
---------------------
3 1 9 | 4 7 5 | 2 6 8
8 5 6 | 1 2 9 | 7 4 3
2 7 4 | 8 3 6 | 1 5 9
---------------------

---------------------
7 9 3 | 6 8 4 | 5 1 2
4 8 6 | 5 1 2 | 9 3 7
1 2 5 | 9 7 3 | 8 4 6
---------------------
9 3 2 | 7 5 1 | 6 8 4
5 7 8 | 2 4 6 | 3 9 1
6 4 1 | 3 9 8 | 7 2 5
---------------------
3 1 9 | 4 6 5 | 2 7 8
8 5 7 | 1 2 9 | 4 6 3
2 6 4 | 8 3 7 | 1 5 9
---------------------

---------------------
6 7 3 | 8 9 4 | 5 1 2
9 1 2 | 7 3 5 | 4 8 6
8 4 5 | 6 1 2 | 9 7 3
---------------------
7 9 8 | 2 6 1 | 3 5 4
5 2 6 | 4 7 3 | 8 9 1
1 3 4 | 5 8 9 | 2 6 7
---------------------
4 6 9 | 1 2 8 | 7 3 5
2 8 7 | 3 5 6 | 1 4 9
3 5 1 | 9 4 7 | 6 2 8
---------------------

